# Exam: Scraping job postings from Jobindex

....

In [216]:
import scraping_class
logfile = 'log_jobindex_scraping.txt' ## name your log file.
connector = scraping_class.Connector(logfile)

In [217]:
import requests
from bs4 import BeautifulSoup
import numpy as np

#define url and fetch the HTML using the requests module
url = 'https://www.jobindex.dk/jobsoegning?lang=da.html'
response = requests.get(url)  
html = response.text  
soup = BeautifulSoup(html,'lxml') # parse the raw html using BeautifoulSoup
#print(soup)

In [218]:
#find the max page number
max_page = soup.find('ul',{'class':'pagination'})
max_page2 = max_page.find_all('a', {'class':'page-link'})

def convert_value_type(value_node):
    if value_node.name == 'a':
        return value_node.text

page_list = []
for page in max_page2:
    page_list.append(convert_value_type(page))
#print(page_list[-1])

last_page = int(page_list[-1]) + 1 # we add one to use it in range in for-loop
print(last_page)

650


In [219]:
#Get all link pages from jobindex
jobindex_links = []
title_company = []
job_data = []
for i in range(1,last_page):
    url = f'https://www.jobindex.dk/jobsoegning?lang=da?page={i}.html'
    jobindex_links.append(url)   
# Get html behind each search page from jobindex
for link in jobindex_links:
    response,call_id = connector.get(link,'searchpage')
    soup = BeautifulSoup(response.text, 'lxml')
    joblistings = soup.find_all('div',{'class':'jobsearch-result'})
# Extract info for each job listing from each searchpage    
    for joblisting in joblistings:
        # find the job title and company which are in bold
        title_and_company = re.findall(r'(?<=<b>)(.*)(?=</b>)', str(joblisting)) 
        title_and_company = title_and_company[0:2]
        soup = BeautifulSoup(str(joblisting), 'lxml')
        # Indirectly extracting the short job description by removing everything else
        for div in soup.find_all("div", {'class':'jix_toolbar jix_appetizer_toolbar'}): # remove toolbar at the end of each job
            div.decompose()
        soup = soup.get_text()
        soup = soup.replace("\n", " ") # remove \n 
        job_info = [str(title_and_company[0]),str(title_and_company[1]),str(soup)] #create a list containing job title, company and description
        job_data.append(job_info)
    
#transforming into dataframe

import pandas as pd
df = pd.DataFrame(job_data)
df.columns = ["Job Title", "Company", "Description"]


# remove job title and company info from description column
df['Description'] = df['Description'].replace(to_replace=r'\b'+df['Job Title']+r'\b', value='',regex=True)
df['Description'] = df['Description'].replace(to_replace=r'\b'+df['Company']+r'\b', value='',regex=True)
df['Description'] = df['Description'].str.strip()

In [221]:
df

,Job Title,Company,Description
0,Forretningsanalytiker med stærk procesforståelse,Udviklings- og Forenklingsstyrelsen,", Frederiksberg Skal du være med til at design..."
1,"Salgskonsulent til sjov, social hverdag",Storm Group,Så er Danmark tilbage efter coronaen! Har du g...
2,Dagplejen Favrskov søger dagplejere,Favrskov Kommune,"Vil du gerne skabe nærvær, yde omsorg og have ..."
3,Regnskabskonsulent,Nomia Roskilde,Har du styr på de klassiske opgaver indenfor b...
4,Salgs- og projektassistent,Orshade Europe ApS,", Lystrup Orshade Europe søger en , som skal ..."
...,...,...,...
12975,Kok – grundet travlhed udvider vi vores køkken...,"Den Gamle Kro, Hornslet 2012 ApS","Vi har brug for en dygtig kok, som vil have mu..."
12976,Det udekørende aftenteam i Stevns Kommune søge...,Stevns Kommune,", Hårlev Kunne du tænke dig at være en del af ..."
12977,Stenslundcentret i Odder søger vicevært,OK-Fonden,Vi søger en vicevært 25 timer om ugen pr. 1. n...
12978,Er du rådgiver og brænder du for at levere løs...,Krüger A/S,"? , Søborg Hvad enten det gælder store mængde..."
